In [2]:

import sys
sys.path.insert(0, '/Users/anis/test_labnbook/Indicators/scripts')  
import json, gzip,ujson
import pandas as pd 
from  utils.utils  import get_authors_changes
import numpy as np, toml
from typing import Tuple, List, Dict
import mysql.connector as mariadb
import warnings    
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',700)

from flow_3 import * 
from utils.utils import *

password = "11950022"
host = "localhost"
database_name  = "Labnbook"
user = 'root'
try:
    conn = mariadb.connect(user=user, password=password,
                           host=host, database=database_name)
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

# Récupération du curseur
cursor = conn.cursor()
trace = pd.read_sql(
    " SELECT id_trace, id_labdoc,id_user ,id_action, action_time from trace Order By id_labdoc ASC, action_time ASC", conn)
# WHERE  id_action=9 

In [91]:
def get_writing_time(id_labdoc : int, trace : pd.DataFrame)-> pd.DataFrame:
    """
    Calculates the number of modification (trace 9) for each change of authors and also multiplies it by 20 or 30 to have the effective writing time 
    output: a list of tuples (id_trace, author_id, num_modifications, writing_time).  
    ex: [(7481881, 12673, 23, 690),
                        (7482763, 12661, 4, 120),
                        (7482817, 12673, 14, 420),
                        (7482845, 12661, 2, 60),
                        [7505160, 12673, 58, 1740]]} 
    """ 
    #TODO: improve this code which is not optimal!
    labdoc  = trace[trace['id_labdoc'] == int(id_labdoc)]
    labdoc.reset_index(drop=True,inplace=True)
    if labdoc.empty:
        pass
    else:
        labdoc_time = labdoc["action_time"].iloc[0]
        date_to_compare = pd.Timestamp('2020-05-16')

        if labdoc_time.date() < date_to_compare.date():
            factor = 20
        else:
            factor = 30
        count_list = []
        current_author = None
        current_count = 0
        for author,id_trace in zip(labdoc['id_user'],labdoc["id_trace"]):
            if author == current_author:
                current_count += 1
            else:
                if current_author is not None:
                    idx  = labdoc[labdoc["id_trace"]==id_trace].index
                    
                    count_list.append((int(labdoc.loc[idx-1].id_trace.values[0]),id_labdoc,current_author, current_count,current_count*factor))
                current_author = author
                current_count = 1
        count_list.append([id_trace,id_labdoc,current_author, current_count,current_count*factor])
        
        return pd.DataFrame(count_list)

In [93]:
id_ld = 445494
count_list = get_writing_time(id_ld, trace)
labdoc = trace[trace["id_labdoc"]==id_ld]
index = get_authors_changes(list(labdoc["id_user"]))
ld = labdoc.iloc[index]
ld 

,id_trace,id_labdoc,id_user,id_action,action_time
7684794,7481881,445494.0,12673,10,2021-03-16 10:05:29
7684798,7482763,445494.0,12661,10,2021-03-16 11:18:11
7684812,7482817,445494.0,12673,10,2021-03-16 11:21:13
7684814,7482845,445494.0,12661,10,2021-03-16 11:22:42
7684872,7505160,445494.0,12673,10,2021-03-18 08:49:40


In [94]:
count_list

,0,1,2,3,4
0,7481881,445494,12673,23,690
1,7482763,445494,12661,4,120
2,7482817,445494,12673,14,420
3,7482845,445494,12661,2,60
4,7505160,445494,12673,58,1740


In [85]:
df1 = pd.DataFrame(count_list)
df2 =df1.head(2)

In [95]:
import pandas as pd

# créer deux dataframes
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})

# concaténer les deux dataframes en ligne
df_concat = pd.concat([df1, df2], axis=0,ignore_index= True)

print(df_concat)


   A  B
0  1  3
1  2  4
2  5  7
3  6  8


In [97]:
df1.append(df2,ignore_index=True)

,A,B
0,1,3
1,2,4
2,5,7
3,6,8


In [6]:
times_df = pd.read_csv('/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_times.csv',index_col=[0])
df_all = pd.read_csv("/Users/anis/test_labnbook/Indicators/data/tmp/reports/3_summary_all.csv",index_col=[0])
df_all.drop("id_labdoc",inplace=True,axis=1)



In [7]:
times_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13920 entries, 0 to 13919
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_trace        13920 non-null  int64  
 1   id_labdoc_x     12889 non-null  float64
 2   id_user         12889 non-null  float64
 3   n_modify_id     12889 non-null  float64
 4   effective_time  12889 non-null  float64
 5   id_labdoc_y     13920 non-null  int64  
 6   action_time     13819 non-null  object 
 7   id_mission      13920 non-null  int64  
 8   id_report       13920 non-null  int64  
 9   n_users         13920 non-null  int64  
 10  teacher         13920 non-null  int64  
 11  n_tokens        13920 non-null  int64  
 12  n_segments      13920 non-null  int64  
 13  eqc             13920 non-null  float64
 14  coec            13920 non-null  float64
 15  user            13920 non-null  object 
 16  sim             13920 non-null  float64
dtypes: float64(7), int64(8), object

In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13920 entries, 0 to 13919
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_trace     13920 non-null  int64  
 1   action_time  13819 non-null  object 
 2   id_mission   13920 non-null  int64  
 3   id_report    13920 non-null  int64  
 4   n_users      13920 non-null  int64  
 5   teacher      13920 non-null  int64  
 6   n_tokens     13920 non-null  int64  
 7   n_segments   13920 non-null  int64  
 8   eqc          13920 non-null  float64
 9   coec         13920 non-null  float64
 10  user         13920 non-null  object 
 11  sim          13920 non-null  float64
dtypes: float64(3), int64(7), object(2)
memory usage: 1.4+ MB


In [9]:
pd.merge(times_df,df_all, on="id_trace", how ="right")


,id_trace,id_labdoc_x,id_user,n_modify_id,effective_time,id_labdoc_y,action_time_x,id_mission_x,id_report_x,n_users_x,...,id_mission_y,id_report_y,n_users_y,teacher_y,n_tokens_y,n_segments_y,eqc_y,coec_y,user_y,sim_y
0,5455308,323357.0,10706.0,18.0,540.0,323357,2020-11-16 09:11:02,451,35694,1,...,451,35694,1,0,32,1,0.0,0.00,10706,1.00
1,5457355,323357.0,10727.0,11.0,330.0,323357,2020-11-16 10:07:17,451,35694,2,...,451,35694,2,0,37,2,0.8,0.15,10727,0.52
2,5461072,323370.0,10727.0,35.0,1050.0,323370,2020-11-16 11:36:42,451,35694,1,...,451,35694,1,0,203,8,0.0,0.00,10727,1.00
3,5462296,323362.0,10727.0,39.0,1170.0,323362,2020-11-16 12:13:46,451,35694,1,...,451,35694,1,0,68,3,0.0,0.00,10727,1.00
4,5459926,323366.0,10727.0,18.0,540.0,323366,2020-11-16 11:09:48,451,35694,1,...,451,35694,1,0,134,5,0.0,0.00,10727,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24015,8356088,473891.0,12099.0,16.0,480.0,473891,2021-05-09 14:00:56,1694,47058,1,...,1694,47058,1,0,45,3,0.0,0.00,12099,1.00
24016,8367872,476308.0,12089.0,4.0,120.0,476308,2021-05-09 23:29:42,1694,47073,1,...,1694,47073,1,0,238,10,0.0,0.00,12089,1.00
24017,8368000,476307.0,12089.0,7.0,210.0,476307,2021-05-09 23:34:29,1694,47073,1,...,1694,47073,1,0,183,10,0.0,0.00,12089,1.00
24018,8367979,476305.0,12089.0,10.0,300.0,476305,2021-05-09 23:33:44,1694,47067,1,...,1694,47067,1,0,183,10,0.0,0.00,12089,1.00


In [115]:
times_df

,id_labdoc,id_trace,action_time,id_mission,id_report,n_users,teacher,n_tokens,n_segments,eqc,coec,user,sim
0,323357,5455308,2020-11-16 09:11:02,451,35694,1,0,32,1,0.0,0.00,10706,1.00
1,323357,5457355,2020-11-16 10:07:17,451,35694,2,0,37,2,0.8,0.15,10727,0.52
2,323370,5461072,2020-11-16 11:36:42,451,35694,1,0,203,8,0.0,0.00,10727,1.00
3,323362,5462296,2020-11-16 12:13:46,451,35694,1,0,68,3,0.0,0.00,10727,1.00
4,323366,5459926,2020-11-16 11:09:48,451,35694,1,0,134,5,0.0,0.00,10727,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,473891,8356088,2021-05-09 14:00:56,1694,47058,1,0,45,3,0.0,0.00,12099,1.00
13916,476308,8367872,2021-05-09 23:29:42,1694,47073,1,0,238,10,0.0,0.00,12089,1.00
13917,476307,8368000,2021-05-09 23:34:29,1694,47073,1,0,183,10,0.0,0.00,12089,1.00
13918,476305,8367979,2021-05-09 23:33:44,1694,47067,1,0,183,10,0.0,0.00,12089,1.00


In [5]:
df_all

,id_trace,action_time,id_mission,id_report,n_users,teacher,n_tokens,n_segments,eqc,coec,user,sim
0,5455308,2020-11-16 09:11:02,451,35694,1,0,32,1,0.0,0.00,10706,1.00
1,5457355,2020-11-16 10:07:17,451,35694,2,0,37,2,0.8,0.15,10727,0.52
2,5461072,2020-11-16 11:36:42,451,35694,1,0,203,8,0.0,0.00,10727,1.00
3,5462296,2020-11-16 12:13:46,451,35694,1,0,68,3,0.0,0.00,10727,1.00
4,5459926,2020-11-16 11:09:48,451,35694,1,0,134,5,0.0,0.00,10727,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
13915,8356088,2021-05-09 14:00:56,1694,47058,1,0,45,3,0.0,0.00,12099,1.00
13916,8367872,2021-05-09 23:29:42,1694,47073,1,0,238,10,0.0,0.00,12089,1.00
13917,8368000,2021-05-09 23:34:29,1694,47073,1,0,183,10,0.0,0.00,12089,1.00
13918,8367979,2021-05-09 23:33:44,1694,47067,1,0,183,10,0.0,0.00,12089,1.00


In [12]:
import numpy as np 


array([0.  , 1.  , 0.75, 0.5 ])

array([0., 1., 1., 1.])

array([0., 0., 0., 0.])

array([0.        , 1.        , 0.5       , 0.66666667])

In [3]:
if not 'x' in locals(): 
    x = 1
    print(x)
else :
    print('x exist')

x exist
